# Workflows

As noted in the article ["Building effective agents" by Anthropic](https://www.anthropic.com/research/building-effective-agents), workflows can be an effective way to use LLMs to complete complex tasks. Our experience building the first iteration of the "note formatting" feature shows that small LLMs cannot complete such a complicated task in one step. This notebook will break down the task into multiple steps of one workflow, which can significantly improve the system's reliability and accuracy.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jinja2 import Template

from anki_ai.domain.deck import Deck
from anki_ai.domain.note import Note
from anki_ai.adapters.chat_completion import get_completion, ChatCompletionsServiceProtocol
from anki_ai.service_layer.services import (
    format_note_workflow,
    add_html_tags,
    remove_html_tags,
)

In [3]:
deck = Deck()
deck.read_txt("../data/Selected Notes v8.txt", exclude_tags=["personal"])
print(f"The deck contains {len(deck)} notes.")

The deck contains 2854 notes.


Let's take a random sample of notes, to iterate more rapidly in our initial exploration. 

In [4]:
deck.shuffle()
sample = deck[:20]  # NOTE: this returs a list, should it instead return a Deck?

Now that we have a small dataset we can use, let's start working on creating those steps in the workflow. 

In [5]:
def compare_notes(original: Note, baseline: Note, challenger: Note) -> None:
    print(f"Front: {note.tags} {remove_html_tags(note.front)}\nBack: {remove_html_tags(note.back)}\n")

    print(f"Front: {baseline.tags} {remove_html_tags(baseline.front)}\nBack: {remove_html_tags(baseline.back)}\n")
    print(f"Front: {challenger.tags} {remove_html_tags(challenger.front)}\nBack: {remove_html_tags(challenger.back)}\n")      
    print("#####################\n")

In [6]:
llm = get_completion()

for note in sample:
    baseline = format_note_workflow(note, llm)  
    challenger = format_note_workflow(note, llm)  # replace this with new challenger
    compare_notes(note, baseline, challenger)

Front: ['nvim'] Motion to move the cursor [count] WORDS forward
Back: `[count]W`

<img src="HYaJn.gif">

Front: ['nvim'] Motion to move the cursor [count] WORDS forward
Back: "[count]W
<img src="HYaJn.gif">"

Front: ['nvim'] Motion to move the cursor [count] WORDS forward
Back: "[count]W
<img src="HYaJn.gif">"

#####################

Front: ['cycling'] What type of tyres can a hooked rim fit?

<img src="hooked-rim-fast-fwd-1.jpeg">
Back: Clincher and (if specified) tubeless-ready tyres

Front: ['cycling'] Hooked rim tyre type
Back: "<img src="hooked-rim-fast-fwd-1.jpeg">
Clincher and (if specified) tubeless-ready tyres"

Front: ['cycling'] Hooked rim tyre type
Back: "<img src="hooked-rim-fast-fwd-1.jpeg">
Clincher and (if specified) tubeless-ready tyres"

#####################

Front: ['linux'] What command line tool controls the Bluetooth  connections?
Back: `bluetoothctl`

Front: ['linux'] Bluetooth connection tool
Back: "`bluetoothctl`"

Front: ['linux'] Bluetooth connection tool
Ba

We are making the LLM process the strings without HTML special characters and tags. The reason for doing this is that we are removing some of the complexity, and making human eval easier.

Another exception is that we make the LLM generate newlines using the HTML tag `<br>`, because we noticed it confuses less the model. When using `\n`, the LLM sometimes starts generating random sequences of symbols.

This new approach seems to lead to better results. There are still minor issues with the recommended changes, that we can try to fix with specialized prompts.

#### Not handled yet:
- [ ] Notes where front or back card contain only an image
- [x] Cloze completions
- [x] Bullet points
- [x] Remove content from `alt` argument in `<img>` tag
- [ ] Code example in front/back that we want to preserve

#### Other things to try:
- [ ] Show note to LLM as it will be shown to user: `[{{ Tags}}] {{ Front }}\n {{ Back }} `
- [x] Do not generate node ID and tags
- [ ] Wrap also front content within double quotes
- [ ] Different diff formats (`whole` --> `diff`). See [Aider's research](https://aider.chat/docs/more/edit-formats.html) on this topic
- [x] Create simple utility function to compare recommendations from different workflows

#### Alternative Design:

1. Original note
1. (Assess note quality)
1. Make concise (input: original note + edited note)
1. Fix code blocks (input: original note + edited note) 
1. Fix math blocks (input: original note + edited note)
1. Fix media (input: original note + edited note)
1. Fix HTML formatting (input: original note + edited note)